## Self-attention Generative Adversarial Capsulet Networks

#### Importing Modules

In [1]:
import os, sys, glob
import numpy as np
import zipfile
import skimage
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras import layers, models, optimizers
from keras import callbacks, backend
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from utils import load_image, load_faces
from discriminator import discriminator_func
from generator import generator_func
from attention import self_attention

/home/nabil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nabil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nabil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nabil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

#### Dataset Preparation

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

root = 'data_faces/img_align_celeba'
img_list = os.listdir(root)
print("Total Number of Images : ", len(img_list))
required_size = (64, 64)



In [ ]:
number_of_images = 50000

def load_image(filename):
  img = Image.open(filename)
  img = image.convert('RGB')
  img = image.resize(required_size)
  pixels = asarray(img)
  pixels = (pixels - 127.5) / 127.5
  return pixels

def load_faces(directory, n_faces):
	faces = list()
	for filename in os.listdir(directory):
		pixels = load_image(directory + filename)
		faces.append(pixels)
		if len(faces) >= n_faces:
			break
	return np.asarray(faces)
 

root = 'data_faces/img_align_celeba/'
dataset = load_faces(directory, number_of_images)
print('Loaded: ', dataset.shape)

In [ ]:
def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors